# EEVE-10.8B QLoRA Fine Tuning

## 1. Install Modules

In [1]:
!pip install -U datasets
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U peft
!pip install -U trl
!pip install -U typing_extensions
!pip install -U torch
!pip install -U datasets
!pip install -U korouge_score
!pip install -U konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 48.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
!pip install ipywidgets
!pip install wandb


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 167.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 156.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 159.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 183.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 169.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 44.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


## 2. Import Modules

In [6]:
import os
import torch
import transformers
from datasets import load_dataset
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    EarlyStoppingCallback,
    Trainer,
    TextStreamer,
    pipeline,
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel,
    PeftConfig,
)
from trl import SFTTrainer
from korouge_score import rouge_scorer
import numpy as np

os.environ["WANDB_PROJECT"] = "FinPilot"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [1]:
from huggingface_hub import login
login()

In [7]:
wandb login

## 3. Load Dataset

In [2]:
from datasets import load_dataset

train_dataset = load_dataset("MartinusChoi/FinPilot", split="train")
test_dataset = load_dataset("MartinusChoi/FinPilot", split="test")

## 4. Set PLM Into QLoRA Form

In [14]:
model_name = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                     # Load model in 4bit precision
    bnb_4bit_quant_type='nf4',             # Pre-trained model has to be quantization in 4bit nf type
    bnb_4bit_use_double_quant=True,        # Use double-qauntization of QLoRA
    bnb_4bit_compute_dtype=torch.bfloat16  # Pre-trained model has to be loaded in BF16 dtype
)

plm = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=True,
    quantization_config=bnb_config,        # Use bitsandbytes config
    device_map='auto',                     # auto : HF Accelerate determines which GPU to allocate for each layer of the model.
    trust_remote_code=True                 # Setting for use EEVE model
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained('yanolja/EEVE-Korean-Instruct-10.8B-v1.0', token=True,)
tokenizer.pad_token = tokenizer.eos_token

## 5. Set Hyper Parameters

In [16]:
flm = prepare_model_for_kbit_training(plm)

lora_alpha = 32
lora_dropout = 0.05
lora_rank = 32

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias='none',
    task_type='CAUSAL_LM',
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj']
)

flm=get_peft_model(flm, peft_config)
flm.print_trainable_parameters()

trainable params: 69,206,016 || all params: 10,874,130,432 || trainable%: 0.6364


In [17]:
from transformers import TrainerCallback

class SaveOnLowerEvalLossCallback(TrainerCallback):
    def __init__(self):
        self.best_eval_loss = float("inf")
    
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        current_loss = metrics.get("eval_loss")
        # eval_loss가 존재하고, 이전보다 낮은 경우에만 저장하도록 설정
        if current_loss is not None and current_loss < self.best_eval_loss:
            self.best_eval_loss = current_loss
            control.should_save = True   # 새로운 최저 loss일 경우 저장 실행
        else:
            control.should_save = False  # 개선되지 않으면 저장하지 않음
        return control

In [18]:
training_arguments = transformers.TrainingArguments(
    output_dir = './train_output',
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 2,                                               # 배치 크기가 줄어들면 기울기 누적 단계가 2배 증가
    optim = 'paged_adamw_32bit',                                                   # 더 나은 메모리 관리를 위해 페이징을 활성화
    save_strategy='steps',                                                         # 학습 중에 채택할 체크포인트 save strategy
    save_steps = 10,                                                               # 두 개의 체크포인트가 저장되기 전의 업데이트 단계 수
    logging_steps = 5,                                                            # 두 로그 사이의 업데이트 단계 수
    learning_rate = 2e-4,                                                          # AdamW 최적화 프로그램의 학습률
    max_grad_norm = 0.3,                                                           # 최대 그라데이션 표준(gradient clipping)
    max_steps = 500,                                                                # 60 단계 동안 학습
    warmup_ratio = 0.03,                                                           # 0 에서 learning_rate 까지 선형 준비에 사용되는 단계 수
    lr_scheduler_type = 'cosine',                                                  # 학습률 스케줄러
    report_to = 'wandb',                                                           # You can find your API key in your browser here: https://wandb.ai/authorize
    evaluation_strategy='steps',  # perform evaluation periodically
    eval_steps=10,                # evaluate every 10 steps
    load_best_model_at_end=True,  # load the best model based on eval_loss
    metric_for_best_model='eval_loss',
    greater_is_better=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## 6. Fine Tuning

In [19]:
def compute_metrics(eval_pred):

    # Dict for store 'rouge' score per predicted summary
    scores = {
        "rouge1": [],
        "rouge2": [],
        "rougeL": [],
        "rougeLsum": []
    }

    # 한국어 rouge score 계산 인스턴스 생성
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'])

    # 모델의 예측 정보 파싱
    predictions, labels = eval_pred

    # 모델 예측의 각 토큰의 대한 logit을 토큰으로 변환
    # prediction shape : (batch, max_length, 51200)
    # 마지막 차원(51200)은 어휘 사전 크기 만큼의 logit
    # np.argmax() 를 활용해 가장 확률 높은 token id로 변환
    predictions = np.argmax(predictions, axis=-1)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # labels에 포함된 -100 값을 pad_token_id로 대체하여 디코딩 에러(OverflowError)를 방지
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    for pred, ref in zip(decoded_preds, decoded_labels):
        score = scorer.score(ref, pred)
        for rouge_type in scores.keys():
            scores[rouge_type].append(score[rouge_type].fmeasure)
    
    avg_scores = {rouge_type: sum(score_list) / len(score_list) * 100 
                  for rouge_type, score_list in scores.items()}
    return avg_scores


In [20]:
trainer = SFTTrainer(
    model=flm,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_arguments,
    compute_metrics=compute_metrics,
    callbacks=[SaveOnLowerEvalLossCallback(), EarlyStoppingCallback(early_stopping_patience=10)]
)


for name, module in trainer.model.named_modules():
    if 'norm' in name:
        module = module.to(torch.float32)

/tmp/ipykernel_6222/645764768.py:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/406 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/406 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/42 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/42 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [21]:
flm.config.use_cache = False
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: martinus-choi (FinPilot) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
10,1.199900,1.191314,45.303187,23.955184,40.997370,44.096028
20,1.078500,1.079117,49.647431,27.214074,45.483246,48.483383
30,1.009400,1.069564,49.854152,27.887221,45.896612,48.814689
40,1.056100,1.065504,50.118459,28.303355,46.020409,48.881543
50,1.031900,1.054673,50.395338,28.891147,46.334466,49.230100
60,0.924600,1.065562,49.702126,27.949690,46.333554,48.860893
70,0.899400,1.050036,50.254053,28.754552,46.474233,49.162414
80,0.784500,1.072389,50.524289,28.655703,46.568255,49.544158
90,0.882800,1.075470,50.019461,28.444443,46.475781,48.958156
100,0.842200,1.090359,49.825245,28.213815,46.313327,48.883944


wandb: Adding directory to artifact (./train_output/checkpoint-10)... Done. 1.0s
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
wandb: Adding directory to artifact (./train_output/checkpoint-20)... Done. 1.0s
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=Tru

TrainOutput(global_step=170, training_loss=0.7680965206202339, metrics={'train_runtime': 1107.9006, 'train_samples_per_second': 1.805, 'train_steps_per_second': 0.451, 'total_flos': 2.662953950104781e+16, 'train_loss': 0.7680965206202339})

## 6. Compare between PLM and FLM

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [23]:
# 예제 입력 텍스트
input_text = """
##Instruction : 삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리잡고 있습니다. 최근 HBM(고대역폭 메모리) 기술 경쟁력을 회복하며 시장에서의 입지를 강화하고 있습니다. 이는 AI 및 데이터 센터 수요 증가에 따른 것으로, 삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 박차를 가하고 있습니다. 이 텍스트를 요약해줘

##Response : 
"""

# FLM 모델 사용
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# 입력 텐서를 GPU로 이동
inputs = {key: value.to(device) for key, value in inputs.items()}

# 모델 실행
output = flm.generate(**inputs, max_length=200)
result = tokenizer.decode(output[0], skip_special_tokens=True)
print("Output:", result)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Output: 
##Instruction : 삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리잡고 있습니다. 최근 HBM(고대역폭 메모리) 기술 경쟁력을 회복하며 시장에서의 입지를 강화하고 있습니다. 이는 AI 및 데이터 센터 수요 증가에 따른 것으로, 삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 박차를 가하고 있습니다. 이 텍스트를 요약해줘

##Response : 
삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리 잡고 있으며, 최근 HBM 기술 경쟁력을 회복하며 시장 입지를 강화하고 있습니다. 이는 AI 및 데이터 센터 수요 증가에 따른 것으로, 삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 집중하고 있습니다.<eos>


In [25]:
# 예제 입력 텍스트
input_text = "삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리잡고 있습니다. 최근 HBM(고대역폭 메모리) 기술 경쟁력을 회복하며 시장에서의 입지를 강화하고 있습니다. 이는 AI 및 데이터 센터 수요 증가에 따른 것으로, 삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 박차를 가하고 있습니다. 이 텍스트를 요약해줘"

# plm 모델 사용
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
# 입력 텐서를 GPU로 이동
inputs = {key: value.to(device) for key, value in inputs.items()}

output = plm.generate(**inputs, max_length=1024)
result = tokenizer.decode(output[0], skip_special_tokens=True)
print("Output:", result)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Output: 삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리잡고 있습니다. 최근 HBM(고대역폭 메모리) 기술 경쟁력을 회복하며 시장에서의 입지를 강화하고 있습니다. 이는 AI 및 데이터 센터 수요 증가에 따른 것으로, 삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 박차를 가하고 있습니다. 이 텍스트를 요약해줘.

삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리 잡고 있으며, 특히 HBM(고대역폭 메모리) 기술 경쟁력을 회복하며 시장에서의 입지를 강화하고 있습니다. 이는 AI 및 데이터 센터 수요 증가에 따른 것으로, 삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 집중하고 있습니다.<eos>


In [27]:
# 예제 입력 텍스트
input_text = """
##Instruction : 삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리잡고 있습니다. 최근 HBM(고대역폭 메모리) 기술 경쟁력을 회복하며 시장에서의 입지를 강화하고 있습니다. 이는 AI 및 데이터 센터 수요 증가에 따른 것으로, 삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 박차를 가하고 있습니다. 이 텍스트를 확장해줘

##Response : 
"""

# FLM 모델 사용
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# 입력 텐서를 GPU로 이동
inputs = {key: value.to(device) for key, value in inputs.items()}

# 모델 실행
output = flm.generate(**inputs, max_length=1024)
result = tokenizer.decode(output[0], skip_special_tokens=True)
print("Output:", result)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Output: 
##Instruction : 삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리잡고 있습니다. 최근 HBM(고대역폭 메모리) 기술 경쟁력을 회복하며 시장에서의 입지를 강화하고 있습니다. 이는 AI 및 데이터 센터 수요 증가에 따른 것으로, 삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 박차를 가하고 있습니다. 이 텍스트를 확장해줘

##Response : 

삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리 잡고 있으며, 이는 회사의 기술력과 시장에서의 입지를 증명하는 중요한 지표입니다. 특히, 최근 삼성전자는 HBM(고대역폭 메모리) 기술 경쟁력을 회복하며 시장에서의 입지를 더욱 강화하고 있습니다. 이 기술은 인공지능(AI) 및 데이터 센터 수요 증가에 따른 것으로, 삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 박차를 가하고 있습니다.

HBM 기술은 고성능 컴퓨팅 시스템에 필수적인 요소로, 대량의 데이터를 빠르게 처리할 수 있는 능력을 제공합니다. 이는 특히 AI와 데이터 센터 분야에서 중요한 역할을 하며, 이러한 시스템은 대량의 데이터를 실시간으로 처리해야 하기 때문입니다. 삼성전자는 HBM 기술을 통해 이러한 시스템의 성능을 향상시키고, 고객들에게 보다 혁신적이고 효율적인 솔루션을 제공하기 위해 노력하고 있습니다.

삼성전자는 HBM 기술 개발에 있어 선도적인 역할을 하고 있으며, 이는 회사의 기술 리더십을 보여주는 중요한 지표입니다. 회사는 지속적인 연구개발을 통해 HBM 기술의 한계를 넘어서는 혁신적인 솔루션을 제공하고 있으며, 이를 통해 시장에서의 입지를 더욱 강화하고 있습니다. 이러한 노력은 삼성전자가 메모리 반도체 분야에서 세계적인 리더로 자리 잡는 데 큰 기여를 하고 있습니다.

삼성전자는 앞으로도 HBM 기술을 비롯한 다양한 메모리 솔루션을 통해 시장에서의 입지를 더욱 강화할 계획입니다. 이를 통해 회사는 고객들에게 보다 혁신적이고 효율적인 솔루션을 제공하며, 메모리 반도체 시장에서 지속적인 성장을 이루

In [24]:
# 예제 입력 텍스트
input_text = "삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리잡고 있습니다. 최근 HBM(고대역폭 메모리) 기술 경쟁력을 회복하며 시장에서의 입지를 강화하고 있습니다. 이는 AI 및 데이터 센터 수요 증가에 따른 것으로, 삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 박차를 가하고 있습니다. 이 텍스트를 확장해줘"

# plm 모델 사용
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
# 입력 텐서를 GPU로 이동
inputs = {key: value.to(device) for key, value in inputs.items()}

output = plm.generate(**inputs, max_length=1024)
result = tokenizer.decode(output[0], skip_special_tokens=True)
print("Output:", result)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Output: 삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리잡고 있습니다. 최근 HBM(고대역폭 메모리) 기술 경쟁력을 회복하며 시장에서의 입지를 강화하고 있습니다. 이는 AI 및 데이터 센터 수요 증가에 따른 것으로, 삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 박차를 가하고 있습니다. 이 텍스트를 확장해줘.

삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리 잡고 있으며, 이는 회사의 핵심 사업 중 하나입니다. 이 회사는 다양한 유형의 메모리 반도체를 생산하고 있으며, 그중에서도 특히 DDR(Double Data Rate) SDRAM, GDDR(Graphics Double Data Rate) SDRAM, LPDDR(Low Power Double Data Rate) SDRAM, 그리고 HBM(High Bandwidth Memory) 등이 주요 제품군에 속합니다. 이러한 제품들은 다양한 전자 제품, 특히 컴퓨터, 스마트폰, 게임 콘솔, 데이터 센터 등에서 필수적인 역할을 합니다.

최근 삼성전자는 HBM 기술 경쟁력을 회복하며 시장에서 입지를 강화하고 있습니다. HBM은 고성능 컴퓨팅 및 그래픽 처리 시스템에 사용되는 고대역폭 메모리 기술로, 대량의 데이터를 빠르게 처리할 수 있는 능력이 요구되는 분야에서의 수요가 증가하고 있습니다. 특히, 인공지능(AI)과 데이터 센터의 수요가 급증하면서 HBM의 중요성이 더욱 부각되고 있습니다.

삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 박차를 가하고 있습니다. 회사는 HBM의 성능을 향상시키기 위해 다양한 기술을 접목하고 있으며, 특히 3D 스택형 구조와 초고속 데이터 전송 기술을 통해 HBM의 한계를 넘어서는 제품을 개발하고 있습니다. 이러한 노력은 삼성전자가 메모리 반도체 시장에서 선도적인 위치를 유지하는 데 중요한 역할을 하고 있습니다.

삼성전자는 또한 HBM의 응용 분야를 확장하기 위해 노력하고 있습니다. 예를 들어, HBM을 데이터 센터의 메모리 솔루션으로 활용하거나, 고성능 그

평가

In [28]:
# 예제 입력 텍스트
input_text = """
##Instruction : 파인텍은 2008년 설립해 끊임없는 R&D 투자와 우수 인력 확보로 전문적인 디스플레이 소재 분야의 선도 기업으로 성장해 왔다. 파인텍은 창립 이래 지속적인 연구 개발로 디스플레이 BLU를 국산화했고, 세계 최초로 폴더블 디스플레이 제조장비를 개발하는 등 디스플레이 산업에서 첨단 기술을 선도하는 기업으로 자리 잡았다. 파인텍은 디스플레이 토털 솔루션 기업을 뛰어넘어 이차전지 장비와 센서 사업에서의 끊임없는 도전과 혁신으로 글로벌 일류 기업으로 위상을 확고히 해 나가고 있다.일진디스플레이의 사업부는 크게 사파이어사업부와 터치사업부로 나뉜다. 사파이어사업부는 발광다이오드(LED)용 사파이어 웨이퍼를 주로 생산하고, 터치사업부는 휴대폰이나 태블릿에 쓰이는 터치스크린 패널을 제조한다. 일진디스플레이의 매출 90% 이상이 터치사업부에서 발생하고, 사파이어사업부의 매출 비중은 10%도 되지 않는다. 하지만 사파이어사업부에 대한 기대는 상당하다. 세계적으로 마이크로 LED 시장이 성장세에 있는데, 마이크로 LED 제조를 위해서는 사파이어 웨이퍼가 필요하기 때문이다. 마이크로 LED는 일반 LED에 비해 길이와 넓이가 각각 10분의 1, 100분의 1 수준이다. 다만, 아직까지는 기술력 등의 문제로 마이크로 LED를 사용하는 기기가 많지 않다. 시장조사기관 마켓앤드마켓은 마이크로 LED 시장 규모가 2020년 17억 달러(약 2조 1,250억 원)에서 2025년 199억 달러(약 25조 원)로 성장할 것으로 전망했다. 마이크로 LED는 스마트워치, 가상현실(VR) 등의 기기를 통해 시장이 형성되면서 향후에는 태블릿, 스마트폰 분야로 확대될 것으로 예상된다. 마이크로 LED 시장이 개화되면 사파이어 소재에 대한 수요가 급격히 늘어날 것으로 전망되며 사파이어 웨이퍼의 가격도 상승할 것으로 기대된다. 이 텍스트를 요약해줘.

##Response : 
"""

# FLM 모델 사용
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# 입력 텐서를 GPU로 이동
inputs = {key: value.to(device) for key, value in inputs.items()}

# 모델 실행
output = flm.generate(**inputs, max_length=1024)
result = tokenizer.decode(output[0], skip_special_tokens=True)
print("Output:", result)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Output: 
##Instruction : 파인텍은 2008년 설립해 끊임없는 R&D 투자와 우수 인력 확보로 전문적인 디스플레이 소재 분야의 선도 기업으로 성장해 왔다. 파인텍은 창립 이래 지속적인 연구 개발로 디스플레이 BLU를 국산화했고, 세계 최초로 폴더블 디스플레이 제조장비를 개발하는 등 디스플레이 산업에서 첨단 기술을 선도하는 기업으로 자리 잡았다. 파인텍은 디스플레이 토털 솔루션 기업을 뛰어넘어 이차전지 장비와 센서 사업에서의 끊임없는 도전과 혁신으로 글로벌 일류 기업으로 위상을 확고히 해 나가고 있다.일진디스플레이의 사업부는 크게 사파이어사업부와 터치사업부로 나뉜다. 사파이어사업부는 발광다이오드(LED)용 사파이어 웨이퍼를 주로 생산하고, 터치사업부는 휴대폰이나 태블릿에 쓰이는 터치스크린 패널을 제조한다. 일진디스플레이의 매출 90% 이상이 터치사업부에서 발생하고, 사파이어사업부의 매출 비중은 10%도 되지 않는다. 하지만 사파이어사업부에 대한 기대는 상당하다. 세계적으로 마이크로 LED 시장이 성장세에 있는데, 마이크로 LED 제조를 위해서는 사파이어 웨이퍼가 필요하기 때문이다. 마이크로 LED는 일반 LED에 비해 길이와 넓이가 각각 10분의 1, 100분의 1 수준이다. 다만, 아직까지는 기술력 등의 문제로 마이크로 LED를 사용하는 기기가 많지 않다. 시장조사기관 마켓앤드마켓은 마이크로 LED 시장 규모가 2020년 17억 달러(약 2조 1,250억 원)에서 2025년 199억 달러(약 25조 원)로 성장할 것으로 전망했다. 마이크로 LED는 스마트워치, 가상현실(VR) 등의 기기를 통해 시장이 형성되면서 향후에는 태블릿, 스마트폰 분야로 확대될 것으로 예상된다. 마이크로 LED 시장이 개화되면 사파이어 소재에 대한 수요가 급격히 늘어날 것으로 전망되며 사파이어 웨이퍼의 가격도 상승할 것으로 기대된다. 이 텍스트를 요약해줘.

##Response : 
파인텍은 2008년 설립된 디스플레이 소재 선도 기업으로, BLU 국산화와 폴더블 디스

In [31]:
# 예제 입력 텍스트
input_text = "파인텍은 2008년 설립해 끊임없는 R&D 투자와 우수 인력 확보로 전문적인 디스플레이 소재 분야의 선도 기업으로 성장해 왔다. 파인텍은 창립 이래 지속적인 연구 개발로 디스플레이 BLU를 국산화했고, 세계 최초로 폴더블 디스플레이 제조장비를 개발하는 등 디스플레이 산업에서 첨단 기술을 선도하는 기업으로 자리 잡았다. 파인텍은 디스플레이 토털 솔루션 기업을 뛰어넘어 이차전지 장비와 센서 사업에서의 끊임없는 도전과 혁신으로 글로벌 일류 기업으로 위상을 확고히 해 나가고 있다.일진디스플레이의 사업부는 크게 사파이어사업부와 터치사업부로 나뉜다. 사파이어사업부는 발광다이오드(LED)용 사파이어 웨이퍼를 주로 생산하고, 터치사업부는 휴대폰이나 태블릿에 쓰이는 터치스크린 패널을 제조한다. 일진디스플레이의 매출 90% 이상이 터치사업부에서 발생하고, 사파이어사업부의 매출 비중은 10%도 되지 않는다. 하지만 사파이어사업부에 대한 기대는 상당하다. 세계적으로 마이크로 LED 시장이 성장세에 있는데, 마이크로 LED 제조를 위해서는 사파이어 웨이퍼가 필요하기 때문이다. 마이크로 LED는 일반 LED에 비해 길이와 넓이가 각각 10분의 1, 100분의 1 수준이다. 다만, 아직까지는 기술력 등의 문제로 마이크로 LED를 사용하는 기기가 많지 않다. 시장조사기관 마켓앤드마켓은 마이크로 LED 시장 규모가 2020년 17억 달러(약 2조 1,250억 원)에서 2025년 199억 달러(약 25조 원)로 성장할 것으로 전망했다. 마이크로 LED는 스마트워치, 가상현실(VR) 등의 기기를 통해 시장이 형성되면서 향후에는 태블릿, 스마트폰 분야로 확대될 것으로 예상된다. 마이크로 LED 시장이 개화되면 사파이어 소재에 대한 수요가 급격히 늘어날 것으로 전망되며 사파이어 웨이퍼의 가격도 상승할 것으로 기대된다. 이 텍스트를 요약해줘."

# plm 모델 사용
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
# 입력 텐서를 GPU로 이동
inputs = {key: value.to(device) for key, value in inputs.items()}

output = plm.generate(**inputs, max_length=1024)
result = tokenizer.decode(output[0], skip_special_tokens=True)
print("Output:", result)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Output: 파인텍은 2008년 설립해 끊임없는 R&D 투자와 우수 인력 확보로 전문적인 디스플레이 소재 분야의 선도 기업으로 성장해 왔다. 파인텍은 창립 이래 지속적인 연구 개발로 디스플레이 BLU를 국산화했고, 세계 최초로 폴더블 디스플레이 제조장비를 개발하는 등 디스플레이 산업에서 첨단 기술을 선도하는 기업으로 자리 잡았다. 파인텍은 디스플레이 토털 솔루션 기업을 뛰어넘어 이차전지 장비와 센서 사업에서의 끊임없는 도전과 혁신으로 글로벌 일류 기업으로 위상을 확고히 해 나가고 있다.일진디스플레이의 사업부는 크게 사파이어사업부와 터치사업부로 나뉜다. 사파이어사업부는 발광다이오드(LED)용 사파이어 웨이퍼를 주로 생산하고, 터치사업부는 휴대폰이나 태블릿에 쓰이는 터치스크린 패널을 제조한다. 일진디스플레이의 매출 90% 이상이 터치사업부에서 발생하고, 사파이어사업부의 매출 비중은 10%도 되지 않는다. 하지만 사파이어사업부에 대한 기대는 상당하다. 세계적으로 마이크로 LED 시장이 성장세에 있는데, 마이크로 LED 제조를 위해서는 사파이어 웨이퍼가 필요하기 때문이다. 마이크로 LED는 일반 LED에 비해 길이와 넓이가 각각 10분의 1, 100분의 1 수준이다. 다만, 아직까지는 기술력 등의 문제로 마이크로 LED를 사용하는 기기가 많지 않다. 시장조사기관 마켓앤드마켓은 마이크로 LED 시장 규모가 2020년 17억 달러(약 2조 1,250억 원)에서 2025년 199억 달러(약 25조 원)로 성장할 것으로 전망했다. 마이크로 LED는 스마트워치, 가상현실(VR) 등의 기기를 통해 시장이 형성되면서 향후에는 태블릿, 스마트폰 분야로 확대될 것으로 예상된다. 마이크로 LED 시장이 개화되면 사파이어 소재에 대한 수요가 급격히 늘어날 것으로 전망되며 사파이어 웨이퍼의 가격도 상승할 것으로 기대된다. 이 텍스트를 요약해줘.

파인텍은 2008년 설립된 디스플레이 소재 선도 기업으로, 디스플레이 BLU 국산화와 폴더블 디스플레이 제조장비 개발에 기여했다. 최근에는 이

In [ ]:
scores = {
    "rouge1": [],
    "rouge2": [],
    "rougeL": [],
    "rougeLsum": []
}

target = "파인텍은 2008년 설립 이후 디스플레이 소재 분야에서 R&D 투자와 인력 확보로 선도 기업으로 성장했으며, 디스플레이 BLU 국산화와 폴더블 디스플레이 제조장비 개발로 주목받고 있다. 또한, 이차전지 장비와 센서 사업에서도 글로벌 기업으로 자리매김하고 있다. 일진디스플레이는 사파이어사업부와 터치사업부로 나뉘며, 매출의 90% 이상이 터치사업부에서 발생한다. 사파이어사업부는 LED용 사파이어 웨이퍼를 생산하며, 마이크로 LED 시장의 성장으로 수요 증가가 기대된다. 마이크로 LED는 스마트워치와 VR 기기에서 시작해 태블릿과 스마트폰으로 확대될 전망이다.<eos>"
pred = "파인텍은 2008년 설립된 디스플레이 소재 선도 기업으로, BLU 국산화와 폴더블 디스플레이 제조장비 개발로 첨단 기술을 선도하고 있다. 최근에는 이차전지 장비와 센서 사업에서도 혁신을 추구하며 글로벌 일류 기업으로 성장하고 있다. 일진디스플레이는 사파이어와 터치스크린 패널을 생산하며, 마이크로 LED 시장의 성장으로 사파이어 웨이퍼에 대한 기대감이 크다. 마이크로 LED 시장은 2025년까지 199억 달러로 성장할 것으로 전망되며, 사파이어 소재 수요도 증가할 것으로 기대된다.<eos>"


rscorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'])
rscore = rscorer.score(target, pred)
for rouge_type in scores.keys():
    scores[rouge_type].append(rscore[rouge_type].fmeasure)
    
avg_scores = {rouge_type: sum(score_list) / len(score_list) * 100 for rouge_type, score_list in scores.items()}

In [30]:
print(avg_scores)

{'rouge1': 44.628099173553714, 'rouge2': 25.21008403361345, 'rougeL': 42.97520661157024, 'rougeLsum': 42.97520661157024}


확장 평가

In [43]:
# 예제 입력 텍스트
input_text = """
##Instruction : 전자랜드는 1988년 국내 최초로 최대 규모의 전자 유통 전문점의 새로운 활로를 개척했다. 전국 130여 개 매장 100%를 직영 중이며, 온라인 쇼핑몰을 운영하고 있다. 국내외 유수 업체의 생활 가전부터 음향/IT/모바일 가전을 넘어, 로봇/드론 등 미래 4차 산업 가전까지 다채로운 상품을 취급한다. 전국 무료 배송, 폐가전 무료 수거, 가전 청소 홈케어 서비스 등 다양한 고객 만족 서비스를 제공하고 있다. 이 텍스트를 확장해줘.

##Response : 
"""

# FLM 모델 사용
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# 입력 텐서를 GPU로 이동
inputs = {key: value.to(device) for key, value in inputs.items()}

# 모델 실행
output = flm.generate(**inputs, max_length=1024)
result = tokenizer.decode(output[0], skip_special_tokens=True)
print("Output:", result)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Output: 
##Instruction : 전자랜드는 1988년 국내 최초로 최대 규모의 전자 유통 전문점의 새로운 활로를 개척했다. 전국 130여 개 매장 100%를 직영 중이며, 온라인 쇼핑몰을 운영하고 있다. 국내외 유수 업체의 생활 가전부터 음향/IT/모바일 가전을 넘어, 로봇/드론 등 미래 4차 산업 가전까지 다채로운 상품을 취급한다. 전국 무료 배송, 폐가전 무료 수거, 가전 청소 홈케어 서비스 등 다양한 고객 만족 서비스를 제공하고 있다. 이 텍스트를 확장해줘.

##Response : 

전자랜드는 1988년에 설립된 국내 최초이자 최대 규모의 전자 유통 전문점으로, 당시 전자 제품 유통 시장에 새로운 활로를 개척한 선구적인 기업으로 평가받고 있습니다. 이 회사는 설립 이래로 꾸준히 성장하며, 현재 전국에 걸쳐 130여 개의 매장을 운영하고 있습니다. 모든 매장은 직영으로 운영되어, 고객들에게 일관된 서비스와 품질을 제공합니다. 

전자랜드는 단순히 제품을 판매하는 데 그치지 않고, 고객들에게 다양한 쇼핑 경험을 제공하기 위해 노력하고 있습니다. 이를 위해 온라인 쇼핑몰을 운영하며, 고객들이 언제 어디서나 편리하게 제품을 구매할 수 있도록 지원하고 있습니다. 온라인 쇼핑몰에서는 다양한 카테고리의 제품을 선보이고 있으며, 고객들이 원하는 제품을 쉽게 찾을 수 있도록 분류하고 있습니다.

전자랜드는 단순한 가전제품 판매에 그치지 않고, 고객의 다양한 요구를 충족시키기 위해 노력하고 있습니다. 이를 위해 국내외 유수 업체의 생활 가전부터 음향/IT/모바일 가전을 취급하고 있으며, 최근에는 로봇과 드론 등 미래 4차 산업 가전까지 제품군을 확장하고 있습니다. 이러한 제품군 확장은 고객의 변화하는 요구를 충족시키고, 새로운 기술 트렌드를 선도하기 위한 전략의 일환입니다.

전자랜드는 단순히 제품을 판매하는 데 그치지 않고, 고객 만족을 위한 다양한 서비스를 제공하고 있습니다. 예를 들어, 전국 무료 배송 서비스를 통해 고객들이 제품을 부담 없이 받을 수

In [45]:
scores = {
    "rouge1": [],
    "rouge2": [],
    "rougeL": [],
    "rougeLsum": []
}

target = """전자랜드는 1988년에 설립되어 국내 최초로 최대 규모의 전자 유통 전문점이라는 새로운 시장을 개척한 기업입니다. 이 회사는 전자 제품 유통 분야에서 선구적인 역할을 하며, 현재 전국에 걸쳐 130여 개의 매장을 운영하고 있습니다. 모든 매장은 100% 직영으로 관리되어, 일관된 서비스와 품질을 고객에게 제공하고 있습니다. 

전자랜드는 오프라인 매장뿐만 아니라 온라인 쇼핑몰도 운영하고 있어, 고객들이 언제 어디서나 편리하게 쇼핑할 수 있는 환경을 제공합니다. 이 온라인 플랫폼은 다양한 전자 제품을 손쉽게 비교하고 구매할 수 있는 기능을 갖추고 있어, 바쁜 현대인들에게 큰 편의를 제공합니다.

취급하는 제품군은 매우 다양합니다. 전통적인 생활 가전 제품에서부터 최신 음향 기기, IT 기기, 모바일 가전 제품에 이르기까지 폭넓은 선택지를 제공합니다. 또한, 로봇과 드론 같은 첨단 기술을 활용한 미래 4차 산업 관련 가전 제품도 취급하고 있어, 기술 발전에 민감한 소비자들의 요구를 충족시키고 있습니다.

전자랜드는 고객 만족을 최우선으로 생각하며, 다양한 부가 서비스를 통해 고객의 편의를 도모하고 있습니다. 전국 어디서나 무료로 배송 서비스를 제공하여, 고객이 원하는 장소에서 제품을 손쉽게 받을 수 있도록 하고 있습니다. 또한, 사용하지 않는 폐가전을 무료로 수거해주는 서비스를 제공하여, 환경 보호에도 기여하고 있습니다. 더불어, 가전 제품의 청소와 유지 관리를 돕는 홈케어 서비스도 제공하여, 고객이 구매한 제품을 오랫동안 최상의 상태로 사용할 수 있도록 지원하고 있습니다.

이러한 다양한 서비스와 제품군을 통해 전자랜드는 고객에게 신뢰받는 브랜드로 자리매김하고 있으며, 끊임없이 변화하는 시장 환경 속에서도 지속적인 성장을 이루고 있습니다. 앞으로도 전자랜드는 혁신적인 제품과 서비스를 통해 고객의 삶을 더욱 풍요롭게 만들기 위해 노력할 것입니다.<eos>"""
pred = """
전자랜드는 1988년에 설립된 국내 최초이자 최대 규모의 전자 유통 전문점으로, 당시 전자 제품 유통 시장에 새로운 활로를 개척한 선구적인 기업으로 평가받고 있습니다. 이 회사는 설립 이래로 꾸준히 성장하며, 현재 전국에 걸쳐 130여 개의 매장을 운영하고 있습니다. 모든 매장은 직영으로 운영되어, 고객들에게 일관된 서비스와 품질을 제공합니다. 

전자랜드는 단순히 제품을 판매하는 데 그치지 않고, 고객들에게 다양한 쇼핑 경험을 제공하기 위해 노력하고 있습니다. 이를 위해 온라인 쇼핑몰을 운영하며, 고객들이 언제 어디서나 편리하게 제품을 구매할 수 있도록 지원하고 있습니다. 온라인 쇼핑몰에서는 다양한 카테고리의 제품을 선보이고 있으며, 고객들이 원하는 제품을 쉽게 찾을 수 있도록 분류하고 있습니다.

전자랜드는 단순한 가전제품 판매에 그치지 않고, 고객의 다양한 요구를 충족시키기 위해 노력하고 있습니다. 이를 위해 국내외 유수 업체의 생활 가전부터 음향/IT/모바일 가전을 취급하고 있으며, 최근에는 로봇과 드론 등 미래 4차 산업 가전까지 제품군을 확장하고 있습니다. 이러한 제품군 확장은 고객의 변화하는 요구를 충족시키고, 새로운 기술 트렌드를 선도하기 위한 전략의 일환입니다.

전자랜드는 단순히 제품을 판매하는 데 그치지 않고, 고객 만족을 위한 다양한 서비스를 제공하고 있습니다. 예를 들어, 전국 무료 배송 서비스를 통해 고객들이 제품을 부담 없이 받을 수 있도록 지원하고 있으며, 폐가전 무료 수거 서비스를 통해 환경 보호에도 기여하고 있습니다. 또한, 가전 청소 홈케어 서비스를 통해 고객들이 제품을 보다 오래 사용할 수 있도록 지원하고 있습니다. 이러한 서비스들은 고객들에게 높은 만족도를 제공하며, 전자랜드의 브랜드 가치를 높이는 데 기여하고 있습니다.

전자랜드는 앞으로도 지속적인 혁신과 고객 중심의 서비스를 통해 전자 제품 유통 시장에서 선도적인 위치를 유지하고자 합니다. 이를 위해 새로운 기술 도입과 고객 맞춤형 서비스 개발에 지속적으로 투자할 계획입니다. 이러한 노력은 고객들에게 보다 나은 쇼핑 경험을 제공하고, 전자랜드가 고객들에게 신뢰받는 브랜드로 자리매김하는 데 기여할 것입니다.<eos>"""


rscorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'])
rscore = rscorer.score(target, pred)
for rouge_type in scores.keys():
    scores[rouge_type].append(rscore[rouge_type].fmeasure)
    
avg_scores = {rouge_type: sum(score_list) / len(score_list) * 100 for rouge_type, score_list in scores.items()}

In [46]:
print(avg_scores)

{'rouge1': 44.44444444444444, 'rouge2': 18.884120171673818, 'rougeL': 32.47863247863248, 'rougeLsum': 37.95309168443497}


In [44]:
# 예제 입력 텍스트
input_text = "전자랜드는 1988년 국내 최초로 최대 규모의 전자 유통 전문점의 새로운 활로를 개척했다. 전국 130여 개 매장 100%를 직영 중이며, 온라인 쇼핑몰을 운영하고 있다. 국내외 유수 업체의 생활 가전부터 음향/IT/모바일 가전을 넘어, 로봇/드론 등 미래 4차 산업 가전까지 다채로운 상품을 취급한다. 전국 무료 배송, 폐가전 무료 수거, 가전 청소 홈케어 서비스 등 다양한 고객 만족 서비스를 제공하고 있다. 이 텍스트를 확장해줘."

# plm 모델 사용
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
# 입력 텐서를 GPU로 이동
inputs = {key: value.to(device) for key, value in inputs.items()}

output = plm.generate(**inputs, max_length=1024)
result = tokenizer.decode(output[0], skip_special_tokens=True)
print("Output:", result)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Output: 전자랜드는 1988년 국내 최초로 최대 규모의 전자 유통 전문점의 새로운 활로를 개척했다. 전국 130여 개 매장 100%를 직영 중이며, 온라인 쇼핑몰을 운영하고 있다. 국내외 유수 업체의 생활 가전부터 음향/IT/모바일 가전을 넘어, 로봇/드론 등 미래 4차 산업 가전까지 다채로운 상품을 취급한다. 전국 무료 배송, 폐가전 무료 수거, 가전 청소 홈케어 서비스 등 다양한 고객 만족 서비스를 제공하고 있다. 이 텍스트를 확장해줘.

전자랜드는 1988년에 설립된 전자 제품 유통 전문점으로, 당시 국내에서는 최초로 대규모 전자 제품 전문점을 운영하며 새로운 유통 채널을 개척한 선구적인 기업으로 평가받고 있습니다. 이 회사는 설립 이래로 꾸준히 성장하여 현재 전국에 걸쳐 130여 개의 매장을 운영하고 있으며, 모든 매장을 직접 운영하는 직영 체제를 유지하고 있습니다. 이러한 직영 체제는 고객들에게 보다 일관되고 신뢰할 수 있는 서비스를 제공하는 데 기여하고 있습니다.

전자랜드는 단순한 전자 제품 판매에 그치지 않고, 온라인 쇼핑몰을 운영하며 온라인 시장에서도 강력한 입지를 다지고 있습니다. 온라인 쇼핑몰을 통해 고객들은 다양한 제품을 쉽게 검색하고 구매할 수 있으며, 이는 고객 편의성을 크게 향상시켰습니다. 또한, 전자랜드는 국내외 유수 업체의 생활 가전 제품을 비롯하여 음향, IT, 모바일 가전을 포함한 다양한 카테고리의 제품을 취급하고 있습니다. 이는 고객들이 다양한 제품을 한 곳에서 비교하고 구매할 수 있는 기회를 제공합니다.

전자랜드는 단순히 제품을 판매하는 데 그치지 않고, 고객 만족을 위한 다양한 서비스를 제공하고 있습니다. 예를 들어, 전국 무료 배송 서비스를 통해 고객들이 제품을 편리하게 받을 수 있도록 지원하며, 폐가전 무료 수거 서비스를 통해 환경 보호에도 기여하고 있습니다. 또한, 가전 청소 홈케어 서비스를 통해 고객들이 가전제품을 보다 청결하게 사용할 수 있도록 돕고 있습니다. 이러한 서비스들은 고객들에게 높은 만족도를 제

In [47]:
scores = {
    "rouge1": [],
    "rouge2": [],
    "rougeL": [],
    "rougeLsum": []
}

target = """전자랜드는 1988년에 설립되어 국내 최초로 최대 규모의 전자 유통 전문점이라는 새로운 시장을 개척한 기업입니다. 이 회사는 전자 제품 유통 분야에서 선구적인 역할을 하며, 현재 전국에 걸쳐 130여 개의 매장을 운영하고 있습니다. 모든 매장은 100% 직영으로 관리되어, 일관된 서비스와 품질을 고객에게 제공하고 있습니다. 

전자랜드는 오프라인 매장뿐만 아니라 온라인 쇼핑몰도 운영하고 있어, 고객들이 언제 어디서나 편리하게 쇼핑할 수 있는 환경을 제공합니다. 이 온라인 플랫폼은 다양한 전자 제품을 손쉽게 비교하고 구매할 수 있는 기능을 갖추고 있어, 바쁜 현대인들에게 큰 편의를 제공합니다.

취급하는 제품군은 매우 다양합니다. 전통적인 생활 가전 제품에서부터 최신 음향 기기, IT 기기, 모바일 가전 제품에 이르기까지 폭넓은 선택지를 제공합니다. 또한, 로봇과 드론 같은 첨단 기술을 활용한 미래 4차 산업 관련 가전 제품도 취급하고 있어, 기술 발전에 민감한 소비자들의 요구를 충족시키고 있습니다.

전자랜드는 고객 만족을 최우선으로 생각하며, 다양한 부가 서비스를 통해 고객의 편의를 도모하고 있습니다. 전국 어디서나 무료로 배송 서비스를 제공하여, 고객이 원하는 장소에서 제품을 손쉽게 받을 수 있도록 하고 있습니다. 또한, 사용하지 않는 폐가전을 무료로 수거해주는 서비스를 제공하여, 환경 보호에도 기여하고 있습니다. 더불어, 가전 제품의 청소와 유지 관리를 돕는 홈케어 서비스도 제공하여, 고객이 구매한 제품을 오랫동안 최상의 상태로 사용할 수 있도록 지원하고 있습니다.

이러한 다양한 서비스와 제품군을 통해 전자랜드는 고객에게 신뢰받는 브랜드로 자리매김하고 있으며, 끊임없이 변화하는 시장 환경 속에서도 지속적인 성장을 이루고 있습니다. 앞으로도 전자랜드는 혁신적인 제품과 서비스를 통해 고객의 삶을 더욱 풍요롭게 만들기 위해 노력할 것입니다.<eos>"""
pred = """
전자랜드는 1988년에 설립된 전자 제품 유통 전문점으로, 당시 국내에서는 최초로 대규모 전자 제품 전문점을 운영하며 새로운 유통 채널을 개척한 선구적인 기업으로 평가받고 있습니다. 이 회사는 설립 이래로 꾸준히 성장하여 현재 전국에 걸쳐 130여 개의 매장을 운영하고 있으며, 모든 매장을 직접 운영하는 직영 체제를 유지하고 있습니다. 이러한 직영 체제는 고객들에게 보다 일관되고 신뢰할 수 있는 서비스를 제공하는 데 기여하고 있습니다.

전자랜드는 단순한 전자 제품 판매에 그치지 않고, 온라인 쇼핑몰을 운영하며 온라인 시장에서도 강력한 입지를 다지고 있습니다. 온라인 쇼핑몰을 통해 고객들은 다양한 제품을 쉽게 검색하고 구매할 수 있으며, 이는 고객 편의성을 크게 향상시켰습니다. 또한, 전자랜드는 국내외 유수 업체의 생활 가전 제품을 비롯하여 음향, IT, 모바일 가전을 포함한 다양한 카테고리의 제품을 취급하고 있습니다. 이는 고객들이 다양한 제품을 한 곳에서 비교하고 구매할 수 있는 기회를 제공합니다.

전자랜드는 단순히 제품을 판매하는 데 그치지 않고, 고객 만족을 위한 다양한 서비스를 제공하고 있습니다. 예를 들어, 전국 무료 배송 서비스를 통해 고객들이 제품을 편리하게 받을 수 있도록 지원하며, 폐가전 무료 수거 서비스를 통해 환경 보호에도 기여하고 있습니다. 또한, 가전 청소 홈케어 서비스를 통해 고객들이 가전제품을 보다 청결하게 사용할 수 있도록 돕고 있습니다. 이러한 서비스들은 고객들에게 높은 만족도를 제공하며, 전자랜드의 브랜드 가치를 높이는 데 기여하고 있습니다.

최근에는 로봇과 드론 등 미래 4차 산업 가전 제품까지 취급 범위를 확장하고 있습니다. 이는 전자랜드가 미래 기술 트렌드를 선도하고, 고객들에게 혁신적인 제품을 제공하려는 의지를 보여줍니다. 이러한 노력은 전자랜드가 앞으로도 지속적으로 성장할 수 있는 기반을 마련하는 데 중요한 역할을 하고 있습니다. 전자랜드는 앞으로도 고객 만족을 최우선으로 생각하며, 다양한 제품과 서비스를 통해 고객들에게 최상의 경험을 제공하고자 노력하고 있습니다.<eos>"""


rscorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'])
rscore = rscorer.score(target, pred)
for rouge_type in scores.keys():
    scores[rouge_type].append(rscore[rouge_type].fmeasure)
    
avg_scores = {rouge_type: sum(score_list) / len(score_list) * 100 for rouge_type, score_list in scores.items()}

In [48]:
print(avg_scores)

{'rouge1': 42.17391304347826, 'rouge2': 17.03056768558952, 'rougeL': 26.086956521739136, 'rougeLsum': 37.391304347826086}


In [34]:
flm.push_to_hub("MartinusChoi/FinPilot-EEVE-Korean-10.8B-1.0.0")

adapter_model.safetensors:   0%|          | 0.00/277M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MartinusChoi/FinPilot-EEVE-Korean-10.8B-1.0.0/commit/a4a89fb991673fff6ef3b32c371ae94473e92615', commit_message='Upload model', commit_description='', oid='a4a89fb991673fff6ef3b32c371ae94473e92615', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MartinusChoi/FinPilot-EEVE-Korean-10.8B-1.0.0', endpoint='https://huggingface.co', repo_type='model', repo_id='MartinusChoi/FinPilot-EEVE-Korean-10.8B-1.0.0'), pr_revision=None, pr_num=None)

In [36]:
peft_model = flm

In [ ]:
# # llama.cpp 클론
# !git clone https://github.com/ggerganov/llama.cpp.git
# !cd llama.cpp && make

# # Python 의존성 설치
# !pip install -r requirements.txt
